In [1]:
from source.regency import Regency
from source.mapping import Mapping
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


seasons = 1
Game = Regency(True)
clear_output()
Game.add_relationship('Go','Fh',At_War=1)

#for a in range(seasons):
Game.random_events()
Game.collect_regency_points()
Game.domain_initiative()
Game.collect_gold_bars()
Game.maintenance_costs(Update=True)
#     Game.Seasons[0][1]['Override']['Go'] = (22,'enemy','Fh')
#     Game.take_domain_actions(1)
#     Game.take_domain_actions(2)
#     Game.take_domain_actions(3)
    # Game.clear_screen()

#df = pd.concat([Game.Seasons[b]['Actions'][a+1] for a in range(3) for b in range(seasons)])
#print(set(df['Action']))
#print()
#print(set(df[df['Success?']==True]['Action']))
#Game.War



Instructions for updating:
Use tf.cast instead.


In [2]:
Game.Seasons[0]['Override'][1]['Go'] = (22,'enemy','Fh')
Game.Seasons[0]['Override'][1]['Fh'] = (20,'friend','FD')
Game.Seasons[0]['Override'][1]['FD'] = (21,'friend','Fh')

In [ ]:
Game.add_relationship('Go','Fh',At_War=1)
Game.take_domain_actions(1)
Game.War

20 Ca 0 0 0 1tion 1 - Initiative 21
20 Ca 0 0 0 1


In [ ]:
Game.Seasons[0]['Override'][2]['FD'] = (21,'friend','Fh')
Game.Seasons[0]['Override'][2]['Go'] = (22,'enemy','FD')
Game.take_domain_actions(2)

In [ ]:
Game.War

In [ ]:
action = 1
Game.Seasons[0]['Actions'][action][Game.Seasons[0]['Actions'][action]['Regent']=='Fh']

In [ ]:
self=Game
Regent='Go'

self.Seasons[self.Season]['Override'][self.Action][Regent]

In [ ]:
Game.attackers

In [ ]:
Game.defenders

In [ ]:
Game.save_forces

In [ ]:
Game.Projects

In [ ]:
Game.Provences[Game.Provences['Regent']=='Fh']